<b>Name: Arpit Aggarwal</b> <br>
<b>UID: 116747189</b>

# 1. Packages

In [238]:
# header files
import numpy as np
import torch
import h5py

# 2. Loading Dataset

Using hw2.ipynb load_data() function. The load_data() function loads data from the training and testing files. Next step, is to flatten the image so that they can be fed as an input to the neural network. Lastly, the training and testing data is normalized between 0 and 1 which will be used for the neural network.

In [252]:
def load_data(train_file, test_file):
    # Load the training data
    train_dataset = h5py.File(train_file, 'r')
    
    # Separate features(x) and labels(y) for training set
    train_set_x_orig = np.array(train_dataset['train_set_x'])
    train_set_y_orig = np.array(train_dataset['train_set_y'])

    # Load the test data
    test_dataset = h5py.File(test_file, 'r')
    
    # Separate features(x) and labels(y) for training set
    test_set_x_orig = np.array(test_dataset['test_set_x'])
    test_set_y_orig = np.array(test_dataset['test_set_y'])
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((test_set_y_orig.shape[0]))
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

# training and testing files
train_file="data/train_catvnoncat.h5"
test_file="data/test_catvnoncat.h5"
train_x_orig, train_y, test_x_orig, test_y, classes = load_data(train_file, test_file) 
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1)
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1)

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten / 255.
test_x = test_x_flatten / 255.

# print data length
print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

train_x's shape: (209, 12288)
test_x's shape: (50, 12288)


# 3. Convert dataset to Tensor form

Convert the dataset to Tensor form so that it can be fed into the PyTorch neural network.

In [253]:
# use torch.from_numpy() to get the tensor form of the numpy array
train_x = torch.from_numpy(train_x).float().to(device)
train_y = torch.from_numpy(train_y).to(device)
train_y = torch.LongTensor(train_y).to(device)
test_x = torch.from_numpy(test_x).float().to(device)
test_y = torch.from_numpy(test_y).to(device)
test_y = torch.LongTensor(test_y).to(device)
train_x.requires_grad = True

# 4. Device Configuration

Set device configuration using pytorch

In [254]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 5. Hyper-parameters

Set the hyper-parameters of the two-layer neural net.

In [255]:
learning_rate = 0.001
num_hidden_neurons = 16
num_epochs = 1000

# 6. Model-Architecture

The model-architecture is defined using pytorch Net class. The __init__ function is where we define the architecture of the neural network, i.e in this it is two layers. The forward function is where the forward pass step of the neural network takes place.

In [256]:
# neural network class
class Net(torch.nn.Module):
    # init function
    def __init__(self, num_input_neurons, num_hidden_neurons, num_output_neurons):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(num_input_neurons, num_hidden_neurons)
        self.fc2 = torch.nn.Linear(num_hidden_neurons, num_output_neurons)
        
    # forward pass step of the neural network
    def forward(self, input):
        input = torch.nn.functional.sigmoid(self.fc2(torch.nn.functional.relu(self.fc1(input))))
        return input
    
# get the neural net object
net = Net(12288, num_hidden_neurons, 1).to(device)
print(net)

Net(
  (fc1): Linear(in_features=12288, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


# 7. Loss function

We will use Cross-entropy loss as we are doing image classification (cat vs non-cat)

In [257]:
# loss function
criterion = torch.nn.CrossEntropyLoss()

# 8. Gradient Descent

Next step is to define the optimizer we will be using for training the neural net. We will use gradient descent as out optimizer.

In [258]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4) 

# 9. Training phase

Now we will be training the neural network to get the optimal set of weights and biases required for this problem.

In [260]:
for epoch in range(0, num_epochs):
    optimizer.zero_grad()
    
    # forward step
    output = net(train_x)
    pred_output = []
    for index in range(0, output.shape[0]):
        class0 = float(1.0 - output[index][0])
        class1 = float(output[index][0])
        pred_output.append(np.array([class0, class1]))
    pred_output = torch.from_numpy(np.array(pred_output)).to(device)
    
    # find loss
    loss = criterion(pred_output, train_y)
    loss = torch.tensor(loss.data, requires_grad=True).to(device)
    
    # backpropagation step
    loss.backward()
    optimizer.step()
    
    if((epoch + 1)%100 == 0):
        print('Loss after iteration {}: {:.4f}' .format(epoch + 1, loss.item()))

/home/arpitdec5/.local/lib/python2.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


Loss after iteration 100: 0.6733
Loss after iteration 200: 0.6733
Loss after iteration 300: 0.6733
Loss after iteration 400: 0.6733
Loss after iteration 500: 0.6733
Loss after iteration 600: 0.6733
Loss after iteration 700: 0.6733
Loss after iteration 800: 0.6733
Loss after iteration 900: 0.6733
Loss after iteration 1000: 0.6733
